In [ ]:
##################################################################
# This code compares the accuracies of various pretrained models.
##################################################################
#################################
# access to google drive
#################################
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#################################
# import ViT pretrained model
#################################
!pip install pytorch_pretrained_vit

  Created wheel for pytorch-pretrained-vit: filename=pytorch_pretrained_vit-0.0.7-py3-none-any.whl size=11132 sha256=af720efeb41e6d6d0e937aad2d485c338ea0969c61ec62c35d1ad3bf5859324a
  Stored in directory: /root/.cache/pip/wheels/87/1d/d1/c6852ef6d18565e5aee866432ab40c6ffbd3411d592035cddb
Successfully built pytorch-pretrained-vit


In [ ]:
#################################
# import modules
#################################
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

import os
import pandas as pd
from torchvision.io import read_image, ImageReadMode
import numpy as np

from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

from pytorch_pretrained_vit import ViT
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.10.0+cu111
Torchvision Version:  0.11.1+cu111


In [ ]:
#################################
# function for model training
#################################
def train_model(model, dataloaders, criterion, optimizer, num_epochs, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    #seoyeon added
    best_epoch = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc

                #seoyeon added
                best_epoch = epoch
                #min_loss = min(val_loss_array)
                #min_loss_epoch = val_loss_array.index(min_loss)

                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'valid':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    #seoyeon added
    print("Best epoch: {}".format(best_epoch))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
#################################
# function to set the requires_grad parameter
#################################
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
#################################
# function to get the model from pytorch models
#################################
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    elif model_name == "inception":
        """ Inception v3 
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299
        
    # *************
    elif model_name == "vit":
        """ Inception v3 
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        # config = dict(hidden_size=512, num_heads=8, num_layers=6)
        # model = ViT.from_config(config)
        model_ft = ViT('B_16_imagenet1k', pretrained=use_pretrained)

        # model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # # Handle the auxilary net
        # num_ftrs = model_ft.AuxLogits.fc.in_features
        # model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 384

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

# **VGG**

In [ ]:
##################################################################
# training and testing VGG model 
##################################################################
data_dir = "/content/drive/MyDrive/comp9417/samples2"
model_name = "vgg"
num_classes = 4
num_epochs = 20
feature_extract = False
batch_size = 43

In [ ]:
#################################
# Initialize the model for this run
#################################
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Print the model we just instantiated
#print(model_ft)

#################################
# Data augmentation and normalization for training
# Just normalization for validation
#################################

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

#print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid', 'test']}
#image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'valid', 'test']}
#dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'valid']}

#################################
# Send the model to GPU
#################################
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)



#################################
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
#################################
params_to_update = model_ft.parameters()
#print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            #print("\t",name)
'''
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            #print("\t",name)
'''
#################################
# Observe that all parameters are being optimized
#################################

optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)


#################################
# Setup the loss fxn
#################################
criterion = nn.CrossEntropyLoss()

#################################
# Train and evaluate
#################################
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/19
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 1.0179 Acc: 0.5926
valid Loss: 0.8517 Acc: 0.6966

Epoch 1/19
----------
train Loss: 0.6089 Acc: 0.8162
valid Loss: 0.5893 Acc: 0.8876

Epoch 2/19
----------
train Loss: 0.4344 Acc: 0.8676
valid Loss: 0.4393 Acc: 0.8989

Epoch 3/19
----------
train Loss: 0.3613 Acc: 0.8794
valid Loss: 0.3827 Acc: 0.9101

Epoch 4/19
----------
train Loss: 0.3543 Acc: 0.8824
valid Loss: 0.3369 Acc: 0.9213

Epoch 5/19
----------
train Loss: 0.2971 Acc: 0.8941
valid Loss: 0.3376 Acc: 0.9213

Epoch 6/19
----------
train Loss: 0.2982 Acc: 0.9029
valid Loss: 0.3553 Acc: 0.9101

Epoch 7/19
----------
train Loss: 0.2754 Acc: 0.8985
valid Loss: 0.3418 Acc: 0.9101

Epoch 8/19
----------
train Loss: 0.2774 Acc: 0.9000
valid Loss: 0.3488 Acc: 0.8876

Epoch 9/19
----------
train Loss: 0.2429 Acc: 0.9118
valid Loss: 0.3816 Acc: 0.8764

Epoch 10/19
----------
train Loss: 0.2159 Acc: 0.9265
valid Loss: 0.3712 Acc: 0.8989

Epoch 11/19
----------
train Loss: 0.2107 Acc: 0.9294
valid Loss: 0.3744 Acc: 0.8876



# **Densenet**

In [ ]:
##################################################################
# training and testing Densenet model 
##################################################################

data_dir = "/content/drive/MyDrive/comp9417/samples2"
model_name = "densenet"
num_classes = 4
num_epochs = 20
feature_extract = False
batch_size = 43

In [ ]:
#################################
# Initialize the model for this run
#################################
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Print the model we just instantiated
#print(model_ft)


#################################
# Data augmentation and normalization for training
# Just normalization for validation
#################################
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

#print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid', 'test']}
#image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'valid', 'test']}
#dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'valid']}

#################################
# Send the model to GPU
#################################
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

#################################
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
#################################
params_to_update = model_ft.parameters()
#print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            #print("\t",name)
'''
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            #print("\t",name)
'''

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)


#################################
# Setup the loss fxn
#################################
criterion = nn.CrossEntropyLoss()

#################################
# Train and evaluate
#################################
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

Epoch 0/19
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 1.1040 Acc: 0.5412
valid Loss: 1.0703 Acc: 0.6404

Epoch 1/19
----------
train Loss: 0.6791 Acc: 0.8176
valid Loss: 0.7214 Acc: 0.8539

Epoch 2/19
----------
train Loss: 0.5028 Acc: 0.8485
valid Loss: 0.4951 Acc: 0.8876

Epoch 3/19
----------
train Loss: 0.3977 Acc: 0.8779
valid Loss: 0.4039 Acc: 0.8989

Epoch 4/19
----------
train Loss: 0.3542 Acc: 0.8779
valid Loss: 0.3860 Acc: 0.8989

Epoch 5/19
----------
train Loss: 0.2912 Acc: 0.9044
valid Loss: 0.3825 Acc: 0.9213

Epoch 6/19
----------
train Loss: 0.2878 Acc: 0.9029
valid Loss: 0.3496 Acc: 0.9213

Epoch 7/19
----------
train Loss: 0.2801 Acc: 0.8926
valid Loss: 0.3435 Acc: 0.9101

Epoch 8/19
----------
train Loss: 0.2922 Acc: 0.8941
valid Loss: 0.3362 Acc: 0.9213

Epoch 9/19
----------
train Loss: 0.2704 Acc: 0.9059
valid Loss: 0.3458 Acc: 0.8989

Epoch 10/19
----------
train Loss: 0.2432 Acc: 0.9044
valid Loss: 0.3351 Acc: 0.9101

Epoch 11/19
----------
train Loss: 0.2273 Acc: 0.9235
valid Loss: 0.3515 Acc: 0.9101



# **VIT**

In [ ]:
##################################################################
# training and testing VIT model 
##################################################################
data_dir = "/content/drive/MyDrive/shared/samples_x384_all_2"
model_name = "vit"
num_classes = 4
num_epochs = 20
feature_extract = False
batch_size = 8

In [ ]:
#################################
# Initialize the model for this run
#################################
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Print the model we just instantiated
#print(model_ft)

#################################
# Data augmentation and normalization for training
# Just normalization for validation
#################################
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

#print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid', 'test']}
#image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'valid', 'test']}
#dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'valid']}

#################################
# Send the model to GPU
#################################
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

#################################
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
#################################
params_to_update = model_ft.parameters()
#print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            #print("\t",name)
'''
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            #print("\t",name)
'''
#################################
# Observe that all parameters are being optimized
#################################
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)


#################################
# Setup the loss fxn
#################################
criterion = nn.CrossEntropyLoss()

#################################
# Train and evaluate
#################################
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Downloading: "https://github.com/lukemelas/PyTorch-Pretrained-ViT/releases/download/0.0.2/B_16_imagenet1k.pth" to /root/.cache/torch/hub/checkpoints/B_16_imagenet1k.pth


  0%|          | 0.00/331M [00:00<?, ?B/s]

Loaded pretrained weights.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 0/19
----------
train Loss: 0.6573 Acc: 0.7603
valid Loss: 0.4316 Acc: 0.8652

Epoch 1/19
----------
train Loss: 0.3818 Acc: 0.8618
valid Loss: 0.3486 Acc: 0.9101

Epoch 2/19
----------
train Loss: 0.3703 Acc: 0.8853
valid Loss: 0.4032 Acc: 0.8427

Epoch 3/19
----------
train Loss: 0.3307 Acc: 0.8912
valid Loss: 0.3033 Acc: 0.9101

Epoch 4/19
----------
train Loss: 0.2973 Acc: 0.8912
valid Loss: 0.3033 Acc: 0.9326

Epoch 5/19
----------
train Loss: 0.3024 Acc: 0.8941
valid Loss: 0.2788 Acc: 0.9101

Epoch 6/19
----------
train Loss: 0.2392 Acc: 0.9206
valid Loss: 0.3286 Acc: 0.9101

Epoch 7/19
----------
train Loss: 0.2151 Acc: 0.9221
valid Loss: 0.3245 Acc: 0.9213

Epoch 8/19
----------
train Loss: 0.2452 Acc: 0.9191
valid Loss: 0.4159 Acc: 0.8876

Epoch 9/19
----------
train Loss: 0.2007 Acc: 0.9250
valid Loss: 0.3033 Acc: 0.8989

Epoch 10/19
----------
train Loss: 0.2116 Acc: 0.9221
valid Loss: 0.2247 Acc: 0.9438

Epoch 11/19
----------
train Loss: 0.1802 Acc: 0.9294
valid Loss

In [ ]:
#################################
# import modules
#################################
from PIL import Image
from torchvision import models
import torch
from torchvision import transforms
import glob
import numpy as np


In [ ]:
#################################
# function to test model on real test sample ( X_test.npy)
#################################
def real_test_model(model, real_test_samples_folder, images_folder, input_size, save_filename):
    
#     with open('imagenet_classes.txt') as f:
#       labels = [line.strip() for line in f.readlines()]
    labels = [0, 1, 2, 3]

    model.eval()

    data_transforms = {
        'real_test': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    } 

    predictions = []
    for filename in glob.glob(f'{real_test_samples_folder}/{images_folder}/*.jpg'): # assuming jpg
        img=Image.open(filename)
        img_t = data_transforms['real_test'](img)
        batch_t = torch.unsqueeze(img_t, 0)
        batch_t = batch_t.to(device)
        out = model(batch_t)
        _, index = torch.max(out, 1)
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        print(labels[index[0]], percentage[index[0]].item())
        predictions.append(labels[index[0]])

    # print('predictions:', predictions)
    np.save(f'{real_test_samples_folder}/{save_filename}', np.array(predictions))

#################################
# validation with known samples: class 3
#################################

real_test_samples_folder = '/content/drive/MyDrive/shared/samples_x384_all/valid'
images_folder = '3'

# input_size = 224 # for other models
input_size = 384 # for ViT
save_filename = 'X_test_preds.npy'

real_test_model(model_ft, real_test_samples_folder, images_folder, input_size, save_filename)
read_again = np.load(f'{real_test_samples_folder}/{save_filename}')
print('read_again', read_again)

1 98.51950073242188
3 77.70064544677734
1 53.738922119140625
3 96.06306457519531
1 92.40181732177734
1 76.2752914428711
3 51.79896926879883
read_again [1 3 1 3 1 1 3]


In [ ]:
#################################
# validation with known samples: class 2
#################################

real_test_samples_folder = '/content/drive/MyDrive/shared/samples_x384_all/valid'
images_folder = '2'

# input_size = 224 # for other models
input_size = 384 # for ViT
save_filename = 'X_test_preds.npy'

real_test_model(model_ft, real_test_samples_folder, images_folder, input_size, save_filename)
read_again = np.load(f'{real_test_samples_folder}/{save_filename}')
print('read_again', read_again)

2 84.23419952392578
2 99.79612731933594
2 68.89125061035156
2 72.36178588867188
2 61.59816360473633
2 99.76758575439453
2 99.48869323730469
2 99.64373016357422
2 94.0243148803711
2 99.85236358642578
0 93.26277160644531
2 94.24295043945312
2 99.67434692382812
2 98.03327178955078
2 99.7677993774414
read_again [2 2 2 2 2 2 2 2 2 2 0 2 2 2 2]


In [ ]:
#################################
# validation with known samples: class 1
#################################

real_test_samples_folder = '/content/drive/MyDrive/shared/samples_x384_all/valid'
images_folder = '1'

# input_size = 224 # for other models
input_size = 384 # for ViT
save_filename = 'X_test_preds.npy'

real_test_model(model_ft, real_test_samples_folder, images_folder, input_size, save_filename)
read_again = np.load(f'{real_test_samples_folder}/{save_filename}')
print('read_again', read_again)

1 99.5134506225586
1 99.39787292480469
1 99.90713500976562
1 99.9311294555664
1 99.45171356201172
1 97.6535873413086
1 99.8643798828125
1 86.0911636352539
1 99.23863220214844
1 99.91548156738281
1 99.88854217529297
1 99.8907470703125
1 99.99181365966797
1 97.93228149414062
1 99.93516540527344
1 98.52837371826172
1 99.71151733398438
1 99.95416259765625
1 98.08396911621094
1 99.92153930664062
1 99.98363494873047
1 98.11920166015625
1 98.71920013427734
read_again [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
#################################
# validation with known samples: class 0
#################################

real_test_samples_folder = '/content/drive/MyDrive/shared/samples_x384_all/valid'
images_folder = '0'

# input_size = 224 # for other models
input_size = 384 # for ViT
save_filename = 'X_test_preds.npy'

real_test_model(model_ft, real_test_samples_folder, images_folder, input_size, save_filename)
read_again = np.load(f'{real_test_samples_folder}/{save_filename}')
print('read_again', read_again)

0 97.57328033447266
0 52.65959548950195
0 99.97470092773438
0 99.75418853759766
0 99.97990417480469
0 99.87649536132812
2 65.33931732177734
0 53.801570892333984
2 52.98597717285156
0 99.75226593017578
0 99.89864349365234
0 99.96532440185547
0 99.94606018066406
0 98.80359649658203
0 99.9292221069336
0 99.8839111328125
0 99.96765899658203
0 97.92407989501953
0 99.98371887207031
0 99.765625
0 99.82606506347656
0 99.78663635253906
0 88.49061584472656
0 99.92247009277344
0 99.29674530029297
0 99.84088897705078
0 99.995361328125
0 99.89934539794922
0 99.99684143066406
0 99.90140533447266
0 99.95480346679688
0 99.90194702148438
0 90.76287078857422
0 99.96986389160156
0 99.99656677246094
0 99.76531982421875
3 43.967716217041016
0 99.97760772705078
0 99.99752044677734
0 99.9637451171875
0 99.9599380493164
0 99.98826599121094
0 99.8092269897461
0 99.40282440185547
read_again [0 0 0 0 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3
 0 0 0 0 0 0 0]


In [ ]:
#################################
# Prediction with unknown samples: X_test.npy
#################################

real_test_samples_folder = '/content/drive/MyDrive/shared/samples_x384_all_real_test'
images_folder = 'real_test'

# input_size = 224 # for other models
input_size = 384 # for ViT
save_filename = 'X_test_preds2.npy'

real_test_model(model_ft, real_test_samples_folder, images_folder, input_size, save_filename)
read_again = np.load(f'{real_test_samples_folder}/{save_filename}')
print('read_again', read_again)

1 79.28813171386719
2 56.013916015625
0 98.91287994384766
1 99.37845611572266
1 99.6634521484375
0 99.84831237792969
1 99.94182586669922
2 93.37516784667969
0 99.9106216430664
0 99.96292877197266
0 99.77616119384766
1 93.9686508178711
1 99.86425018310547
2 99.45203399658203
1 99.94422912597656
1 99.94448852539062
0 99.98042297363281
0 99.87792205810547
2 99.56110382080078
0 99.99675750732422
0 99.9860610961914
2 61.75516128540039
2 99.7755355834961
2 99.7737045288086
2 99.53882598876953
0 99.78753662109375
0 99.96611022949219
0 96.11784362792969
0 99.99434661865234
0 99.9969711303711
0 99.98304748535156
0 99.95040893554688
2 99.86946868896484
2 99.89982604980469
0 99.86529541015625
0 91.61483764648438
1 97.03882598876953
2 98.92642974853516
1 99.72755432128906
0 63.662357330322266
1 99.58810424804688
0 99.98643493652344
0 98.33781433105469
0 99.99768829345703
2 94.37720489501953
1 95.7656021118164
1 85.60391998291016
2 99.61555480957031
0 76.69023132324219
0 99.98749542236328
1 99.2483